# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:

* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with Tensorflow (or TPUs)
* Exercises to try for yourself

## Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2023-05-31 23:07:29.517648: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 23:07:29.770021: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 23:07:29.771156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 23:07:30.997438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


In [2]:
# Create a tensor with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
print(f'Number of dimensions: {scalar.ndim}')

Number of dimensions: 0


In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimensions of our vector
print(f'Number of dimensions: {vector.ndim}')

Number of dimensions: 1


In [6]:
# Create a matrix (has more one dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])

matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# Check dimensions of our matrix
print(f'Number of dimensions: {matrix.ndim}')

Number of dimensions: 2


In [8]:
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16)  # specify data type with dtype parameter

another_matrix
# shape(row, columns)

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# Number of dimensions of another matrix
print(f'Number of dimensions: {another_matrix.ndim}')

Number of dimensions: 2


In [10]:
# Let's create a tensor
tensor = tf.constant([[[1,2,3],
                      [4,5,6]],
                     [[7,8,9],
                     [10,11,12]],
                     [[13, 14,15],
                     [16, 17,18]]])

tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
# Number of dimensions of our tensor
print(f'Number of dimensions: {tensor.ndim}')

Number of dimensions: 3


### What we've created so far:

* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

## Create tensors with tf.Variable()

In [12]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [13]:
# Create the same tensor with tf.Variable() as above
changable_tensor = tf.Variable([10, 7])
unchangable_tensor = tf.constant([10, 7])

In [14]:
changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [15]:
# Let's try change one of the elements in our changable tensor
changable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [16]:
changable_tensor[0] = 7

TypeError: 'ResourceVariable' object does not support item assignment

In [17]:
# How about we try .assing()
changable_tensor[0].assign(7)
changable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [18]:
# Let's try change our unchangeable tensor
unchangable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [19]:
unchangable_tensor[0] = 7

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [20]:
unchangable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

## Creating random tensors

Random tensors are tensors of some arbitrary size wich contain random nunmbers.

In [21]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)  # Set seed for reproducibility
random_1 = random_1.normal(shape=(3, 2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [22]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [23]:
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [24]:
random_3 = tf.random.Generator.from_seed(7)
random_3 = random_3.normal(shape=(3, 2))

random_4 = tf.random.Generator.from_seed(7)
random_4 = random_4.normal(shape=(3, 2))

random_3, random_4, random_3 == random_4

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

## Shuffle the order of elements in a tensor

In [25]:
# Shuffle the tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2, 5]])

not_shuffled.ndim

2

In [26]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [27]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)  # Random shuffles a tensor along its FIRST DIMENSION

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [28]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [29]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [30]:
tf.random.set_seed(42)  # Global level seed
tf.random.shuffle(not_shuffled, seed=42)  # Operation level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

Isso é conhecido como um tensor de classificação 3 (3 dimensões), no entanto, um tensor pode ter uma quantidade arbitrária (ilimitada) de dimensões.

Por exemplo, você pode transformar uma série de imagens em tensores com forma (224, 224, 3, 32), onde:
* 224, 224 (as primeiras 2 dimensões) são a altura e a largura das imagens em pixels.
* 3 é o número de canais de cores da imagem (vermelho, verde, azul).
* 32 é o tamanho do lote (o número de imagens que uma rede neural vê a qualquer momento).

Todas as variáveis ​​acima que criamos são, na verdade, tensores. Mas você também pode ouvi-los referidos por seus nomes diferentes (os que demos a eles):
* **escalar**: um único número.
* **vetor**: um número com direção (por exemplo, velocidade do vento com direção).
* **matriz**: uma matriz bidimensional de números.
* **tensor**: uma matriz n-dimensional de números (onde n pode ser qualquer número, um tensor de dimensão 0 é um escalar, um tensor de dimensão 1 é um vetor).

Para aumentar a confusão, os termos matriz e tensor são frequentemente usados ​​de forma intercambiável.

Daqui para frente, já que estamos usando o TensorFlow, tudo o que nos referirmos e usarmos serão tensores.

Para saber mais sobre a diferença matemática entre escalares, vetores e matrizes, consulte a [postagem de álgebra visual de Math is Fun](https://www.mathsisfun.com/algebra/scalar-vector-matrix.html).

![diferença entre escalar, vetor, matriz, tensor](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/00-scalar-vector-matrix-tensor.png)

🛠 **Exercise:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [31]:
tf.random.set_seed(4029)

tensor1 = tf.random.uniform(shape=(2,2), seed=7) 
tensor2 = tf.random.uniform(shape=(2,2), seed=7)
tensor3 = tf.random.uniform(shape=(2,2), seed=7)
tensor4 = tf.random.uniform(shape=(2,2), seed=7)
tensor5 = tf.random.uniform(shape=(2,2), seed=7)

tensor1, tensor2, tensor3, tensor4, tensor5

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.17115712, 0.6531501 ],
        [0.8587817 , 0.874786  ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.7144613 , 0.37347782],
        [0.641343  , 0.10519004]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.2770716 , 0.5369843 ],
        [0.34588027, 0.60669184]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.5720482 , 0.18713129],
        [0.19723058, 0.462242  ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.2036438 , 0.44336116],
        [0.07702982, 0.71694434]], dtype=float32)>)

In [32]:
shuffled1 = tf.random.shuffle(tensor1)
shuffled2 = tf.random.shuffle(tensor2)
shuffled3 = tf.random.shuffle(tensor3)
shuffled4 = tf.random.shuffle(tensor4)
shuffled5 = tf.random.shuffle(tensor5)

shuffled1, shuffled2, shuffled3, shuffled4, shuffled5

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.8587817 , 0.874786  ],
        [0.17115712, 0.6531501 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.7144613 , 0.37347782],
        [0.641343  , 0.10519004]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.34588027, 0.60669184],
        [0.2770716 , 0.5369843 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.19723058, 0.462242  ],
        [0.5720482 , 0.18713129]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.07702982, 0.71694434],
        [0.2036438 , 0.44336116]], dtype=float32)>)

## Other ways to make tensors

In [33]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [34]:
# Create a tensor of all zeros
tf.zeros([3, 4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [35]:
import numpy as np

In [36]:
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

# X = tf.constant(some_matrix) -> Capital for Matrix
# y = tf.constant(vector) -> Non-Capital for Vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [37]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [38]:
A = tf.constant(numpy_A, shape=(2,3,4))
B = tf.constant(numpy_A)

A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [39]:
2 * 3 * 4

24

In [40]:
# shape=(2, 3, 4) -> 2 * 3 * 4 == len(numpy_A) -> Number of elements in numpy_A
print((2 * 3 * 4) == len(numpy_A))

True


In [41]:
C = tf.constant(numpy_A, shape=(2, 3, 5))

TypeError: Eager execution of tf.constant with unsupported shape. Tensor [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] (converted from [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]) has 24 elements, but got `shape` (2, 3, 5) with 30 elements).

In [42]:
D = tf.constant(numpy_A, shape=(3, 8))
3 * 8, D

(24,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>)

In [43]:
A.ndim, B.ndim, D.ndim

(3, 1, 2)


## Getting information from tensors

When dealing with tensors, you probably want to be aware of the following attributes:

* Shape → The length (number of elements) of each of dimension of a tensor.
  <br><br>`tensor.shape`<br><br>
* Rank → The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix is rank 2, a tensor has rank n.
  <br><br>`tensor.ndim`<br><br>
* Axis of dimension → A particular dimension of a tensor.
  <br><br>`tensor[0], tensor[:, 1]`<br><br>
* Size → The total number of items in the tensor.
  <br><br>`tf.size(tensor)`

In [44]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [45]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [46]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [47]:
2 * 3 * 4 * 5

120

In [48]:
# Get various attributes of our tensor
print(f'Datatype of every element: {rank_4_tensor.dtype}')
print(f'Number of dimensions (rank): {rank_4_tensor.ndim}')
print(f'Shape of tensor: {rank_4_tensor.shape}')
print(f'Elements along the 0 axis: {rank_4_tensor.shape[0]}')
print(f'Elemenst along the last axis: {rank_4_tensor.shape[-1]}')
print(f'Total number of elements in our tensor: {tf.size(rank_4_tensor)}')

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elemenst along the last axis: 5
Total number of elements in our tensor: 120


## Indexing Tensors

Tensors can be indexing just like pyhton lists.

In [49]:
some_list = [1,2,3,4]
some_list[:2]

[1, 2]

In [50]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
some_list[:1]

[1]

In [52]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [53]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [54]:
rank_4_tensor[:1, :1, :1,]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [55]:
rank_4_tensor[:1, :1, :1, :1]

<tf.Tensor: shape=(1, 1, 1, 1), dtype=float32, numpy=array([[[[0.]]]], dtype=float32)>

In [56]:
rank_4_tensor[:, :1, :1, :1]

<tf.Tensor: shape=(2, 1, 1, 1), dtype=float32, numpy=
array([[[[0.]]],


       [[[0.]]]], dtype=float32)>

In [57]:
rank_4_tensor[:1, :, :1, :1]

<tf.Tensor: shape=(1, 3, 1, 1), dtype=float32, numpy=
array([[[[0.]],

        [[0.]],

        [[0.]]]], dtype=float32)>

In [58]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [59]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                            [3, 4]])

rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [60]:
rank_2_tensor.shape, rank_2_tensor.ndim, tf.size(rank_2_tensor)

(TensorShape([2, 2]), 2, <tf.Tensor: shape=(), dtype=int32, numpy=4>)

In [61]:
some_list, some_list[-1]

([1, 2, 3, 4], 4)

In [62]:
# Get the last item if each of row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [63]:
# Add in extra dimension to our rank 2 tensor

rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [64]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1)  # '-1' means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [65]:
tf.expand_dims(rank_2_tensor, axis=1)

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

## Munipulating tensor (tensor operations)

### Basic operations

`+`, `-`, `*`, `/`

In [66]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [67]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [68]:
# Change tensor
tensor = tensor + 10
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [69]:
tensor -= 10 
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [70]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [71]:
# Substraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [72]:
tensor / 2

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[5. , 3.5],
       [1.5, 2. ]])>

In [73]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10)  # Speed up on a GPU

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [74]:
tf.add(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [75]:
tf.divide(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

In [76]:
tf.subtract(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

### Matrix Multiplication

In machine learning, matrix multiplication is one of the ,ost common tensor operations.

There are two rules our tensors (or matrices) need to fulfil if we're going to matrix mutiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [77]:
# Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [78]:
tensor, tensor

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>)

In [79]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [80]:
# Matrix multiplication with Python operator '@'
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [81]:
tensor.shape

TensorShape([2, 2])

In [82]:
# Create a tensor (3, 2) tensor
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])

# Create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [83]:
# Try to matrix multiply tensors of same shape
X @ Y  # Number of columns of X must equal number of rows of Y

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

In [85]:
tf.matmul(X, Y)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

📖**Resource**: Info and exmaple of matrix multiplication: https://www.mathsisfun.com/algebra/matrix-multiplying.html

In [86]:
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [87]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [88]:
X.shape, tf.reshape(Y, shape=(3, 2)).shape

(TensorShape([3, 2]), TensorShape([3, 2]))

In [89]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [90]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [91]:
# Try change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [92]:
tf.reshape(X, shape=(2, 3)).shape, Y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

In [93]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [94]:
# Can do the same with transpose
tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [95]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [96]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [97]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [98]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [99]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [100]:
# Check the values of Y, reshape Y and transpose Y
print('Normal Y:')
print(Y, '\n')
print('Y reshaped to (2, 3):')
print(tf.reshape(Y, (2, 3)))
print('Y transposed:')
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)
Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [101]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Generally, when performing matrix multiplication on two tensors and one of axes doesn't line up, you will transpose (rather than reshape) one of the tensors to get satisify the matrix multiplication rules.

## Changing the datatype of tensors

In [102]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])  # The default datatype will depend on what data is inside your tensor
B.dtype

tf.float32

In [103]:
C = tf.constant([7, 10])
C

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

📚 Mixed Precision TF: https://www.tensorflow.org/guide/mixed_precision?hl=pt-br

In [104]:
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [105]:
# Change from float32 to float16 (reduce precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [106]:
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>,
 tf.int32)

In [107]:
E = tf.cast(C, dtype=tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [108]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16, E_float16.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>,
 tf.float16)

## Aggregating tensors

Aggregating tensors = Condensing them from multiple values down to smaller amount of values.

In [109]:
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [110]:
# Get the absolute values
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through following forms of aggrefation:

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [111]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([51, 88, 58, 51, 55, 91,  3, 77, 96, 76, 34, 82, 71, 91, 49, 35, 27,
       61, 97, 98, 55,  0, 35, 37, 65, 17, 36, 77, 19, 85, 13, 80, 13, 36,
       10, 52, 25,  9, 44, 88, 48, 50, 12, 38, 23, 40, 92, 92, 61, 45])>

In [112]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [113]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [114]:
tf.reduce_min(E) == np.min(E)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [115]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=98>

In [116]:
tf.reduce_max(E) == np.max(E)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [117]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=51>

In [118]:
tf.reduce_mean(E) == np.mean(E)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [119]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2588>

In [120]:
tf.reduce_sum(E) == np.sum(E)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

🛠 **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using TensorFlow methods

In [121]:
E 

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([51, 88, 58, 51, 55, 91,  3, 77, 96, 76, 34, 82, 71, 91, 49, 35, 27,
       61, 97, 98, 55,  0, 35, 37, 65, 17, 36, 77, 19, 85, 13, 80, 13, 36,
       10, 52, 25,  9, 44, 88, 48, 50, 12, 38, 23, 40, 92, 92, 61, 45])>

In [122]:
E = tf.cast(E, dtype=tf.float32)  # With int doesn't work

In [123]:
E

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([51., 88., 58., 51., 55., 91.,  3., 77., 96., 76., 34., 82., 71.,
       91., 49., 35., 27., 61., 97., 98., 55.,  0., 35., 37., 65., 17.,
       36., 77., 19., 85., 13., 80., 13., 36., 10., 52., 25.,  9., 44.,
       88., 48., 50., 12., 38., 23., 40., 92., 92., 61., 45.],
      dtype=float32)>

In [124]:
# Variance of tensor
tf.math.reduce_variance(E)

<tf.Tensor: shape=(), dtype=float32, numpy=804.9024>

In [125]:
# Standard Deviation(STD) of tensor
tf.math.reduce_std(E)

<tf.Tensor: shape=(), dtype=float32, numpy=28.370802>

In [126]:
import tensorflow_probability as tfp

In [127]:
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=float32, numpy=804.9024>

In [128]:
tfp.stats.stddev(E)

<tf.Tensor: shape=(), dtype=float32, numpy=28.370802>

## Find the positional maximum and minimum

In [129]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [130]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [131]:
tf.argmax(F) == np.argmax(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [132]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [133]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [134]:
tf.reduce_max(F) == F[tf.argmax(F)] == F[np.argmax(F)]

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [135]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [136]:
# Index on our smaller value position
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [137]:
tf.reduce_min(F) == F[tf.argmin(F)] == F[np.argmin(F)]

<tf.Tensor: shape=(), dtype=bool, numpy=True>

## Squeezing a tensor (removing all single dimensions)

In [138]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [139]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [140]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

## One-hot encoding tensors

One-hot encoding is a form of numerical encoding 

In [141]:
# Create some list of indices
some_list = [0, 1, 2, 3]  # Could be red, green, blue and purple

In [142]:
# One-hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [143]:
tf.one_hot(some_list, depth=3)

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

In [144]:
# Specify custom values for one-hot encoding
tf.one_hot(some_list, depth=4, on_value='On', off_value='Off')

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'On', b'Off', b'Off', b'Off'],
       [b'Off', b'On', b'Off', b'Off'],
       [b'Off', b'Off', b'On', b'Off'],
       [b'Off', b'Off', b'Off', b'On']], dtype=object)>

## Squaring, log and Square Root

In [145]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [146]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [147]:
# Find the square root
tf.sqrt(H)  # Will error, method requires non-int type

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sqrt}}; Op<name=Sqrt; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sqrt]

In [148]:
# Find the square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [149]:
# Find the log
tf.math.log(H)  # Will error, method requires non-int type

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Log}}; Op<name=Log; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Log]

In [ ]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

## Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays.

🔑 **Note:** One of the main differences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster numerical processing)

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

In [150]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

NameError: name 'J' is not defined

In [151]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

NameError: name 'J' is not defined

In [152]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

In [153]:
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

## Finding access to GPUs

In [154]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [155]:
tf.config.list_physical_devices('GPU')

[]

In [156]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


🔑 **Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenver possible.